# El corrido de Gregorio Cortez

## Mapping the manhunt of Rio Grande border folk hero Gregorio Cortez:  the largest manhunt in U.S. history.

### June 14, 1901 to June 22, 1901

### based on lyrics from the Corrido de Gregorio Cortez (variant x english translation by Americo Paredes)

#### project author Diane Lopez

In [43]:
import voila
import geopandas as gpd

import geograpy
from geograpy import extraction
from geograpy import places
import re
from geograpy.labels import Labels
#from geograpy import geograpy-nltk

import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

import pandas as pd
import numpy as np

import shapely
from shapely.geometry import Point

import contextily as ctx
import matplotlib.pyplot as plt #to make sure there are no errors when plotting a graph
import plotly.express as px
import plotly.graph_objects as go

#from mordecai import Geoparser
import nltk

import spacy
from spacy import displacy
#from spacy.lang.en import English
#from spacy.lang.es import Spanish
#from spacy.pipeline import EntityRuler
#from spacy.tokens import Doc
#from spacy.training import Example
#from spacy.language import Language

import json
import random

import locationtagger
nlp = spacy.load("en_core_web_sm")

In [23]:

text = "corrido corpus\ElCorridodeGregorioCortez_X.txt"
with open(text, 'r', encoding='utf-8') as c:
    text = c.read()
    
def clean_text(text):
    cleaned= re.sub(r'[":;,.“”]', "", text)
    return(cleaned)
text = clean_text(text)
#print(text)


TxGPE=[]
nlp = spacy.load("tx_ner_model")
doc =nlp(text) 
#print(doc)
for ent in doc.ents:
    #print(ent.text, ent.label_)
    if ent.label_ == "GPE":
        TxGPE.append(ent.text)
print(TxGPE)

['Karnes', 'Gonzales', 'Belmont', 'Laredo', 'Encinal']


In [24]:
ents = [(e.text, e.start_char, e.end_char, e.label_)for e in doc.ents]
print(ents)

ents = [(e.text)for e in doc.ents]
print(ents)

[('Karnes', 18, 24, 'GPE'), ('Gonzales', 1051, 1059, 'GPE'), ('Belmont', 1140, 1147, 'GPE'), ('Laredo', 1550, 1556, 'GPE'), ('Encinal', 1864, 1871, 'GPE')]
['Karnes', 'Gonzales', 'Belmont', 'Laredo', 'Encinal']


In [5]:
displacy.render(doc, style ='ent', jupyter=True, page=True)

Geoparsing: Finding places from the corrido lyrics

In [22]:
pc = places.PlaceContext(ents)

pc.set_countries()
print (pc.countries) #['United States']

pc.set_regions()
print(pc.regions) #['Texas'])

pc.set_cities()
print(pc.cities) #['Brownsville'])

#print(pc.address_strings) #['Brownsville','Texas, United States'])
print(pc.cities[0:5])

[]
[]
['Laredo', 'Belmont', 'Gonzales', 'Encinal', 'Karnes']
['Laredo', 'Belmont', 'Gonzales', 'Encinal', 'Karnes']


need to know the country code https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2#Current_codes

Setting Geometry- https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.set_geometry.html#

Creating a GeoDataFrame from a DataFrame with coordinates https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html

Pandas Dataframe

In [83]:
df = pd.DataFrame(ents, columns=["NER Places"])
df.head()


,NER Places
0,Karnes
1,Gonzales
2,Belmont
3,Laredo
4,Encinal


NER Places Geolocation 

In [84]:
geolocator = Nominatim(user_agent="https://nominatim.openstreetmap.org/search?place=hamlet")

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

df['Location'] = df['NER Places'].apply(geocode)

df.head()

,NER Places,Location
0,Karnes,"(Karnes County, Texas, United States, (28.8791..."
1,Gonzales,"(Gonzales County, Texas, United States, (29.44..."
2,Belmont,"(Belmont, Molsheim, Bas-Rhin, Grand Est, Franc..."
3,Laredo,"(Laredo, Webb County, Texas, United States, (2..."
4,Encinal,"(Encinal, La Salle County, Texas, United State..."


In [ ]:
laredo_info = geolocator.geocode(laredo)
laredo_lat = (laredo_info.latitude) 
laredo_long= (laredo_info.longitude)
print(laredo_info)
print(laredo_long,',', laredo_lat)

Belmont, Tejas/Texas

In [ ]:
belmont='Belmont, Gonzales County,Texas'
#belmont=pc.cities[1]
belmont_info=geolocator.geocode(belmont)
belmont_lat = (belmont_info.latitude)
belmont_long= (belmont_info.longitude)
print(belmont_info)
print(belmont_long,',', belmont_lat)

Gonzales, Tejas

In [ ]:

#gonzales='Gonzales,Texas'
gonzales=pc.cities[2]
gonzales_info=geolocator.geocode(gonzales)
gonzales_lat = (gonzales_info.latitude) 
gonzales_long =(gonzales_info.longitude)
#gonzales=gonzales_info, gonzales_coordinates
print(gonzales_lat, ',', gonzales_long)
print(gonzales_info)

In [ ]:
greg_gpd = gpd.GeoDataFrame(
    nerPlaces, geometry=gpd.points_from_xy(nerPlaces.Longitude, nerPlaces.Latitude),  crs="EPSG:4326")

figure out later shapely 2.0 

In [ ]:
print(greg_gpd.head())
#greg_gpd.plot()

In [ ]:
#greg_gpd_basemap, greg_gpd_extent = ctx.bounds2img(*greg_gpd.total_bounds, zoom=10,   
                                            #source=ctx.providers.OpenStreetMap.Mapnik )

In [ ]:
px.set_mapbox_access_token(open("mapboxtoken").read())

fig = px.scatter_mapbox(greg_gpd, 
                        color='Place',
                        animation_group='Place',
                        animation_frame=['Gonzales', 'Belmont', 'Laredo'],
                        lat=greg_gpd.geometry.y, 
                        lon=greg_gpd.geometry.x, 
                        hover_name='Place',
                        size='Latitude',
                        zoom=5,
                        width=500,
                        height=500,
                        title=("Gregorio Cortez on the run (June 14 to 22, 1901)"),
                        mapbox_style='carto-positron'                               
                        )

fig.show()

In [ ]:
greg_gpd.to_json()

In [ ]:
import os
greg_gpd.to_file(r"C:\Users\dmlpz\Corridos", driver="GeoJSON")

In [ ]:
fig.write_html("../corridos/gregmap.html")